## LLM Evaluation

In [ ]:
#pip install evaluate

In [1]:
!pip install evaluate rouge_score --quiet

In [2]:
prediction = ["Today is Wednesday and I am enjoying it here.".lower()]
actual = ["Today is wednesday and I want to travel somwhere.".lower()]

In [3]:
import evaluate
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=prediction,references=actual)
print(results)

/home/user/GAAPB03-training/gen-ai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.

{'bleu': 0.4111336169005197, 'precisions': [0.6, 0.4444444444444444, 0.375, 0.2857142857142857], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 10, 'reference_length': 10}


In [4]:
rouge = evaluate.load("rouge")
results = rouge.compute(predictions=prediction,references=actual)
print(results)

{'rouge1': 0.5555555555555556, 'rouge2': 0.5, 'rougeL': 0.5555555555555556, 'rougeLsum': 0.5555555555555556}


In [5]:
meteor = evaluate.load("meteor")
results = meteor.compute(predictions=prediction,references=actual)
print(results)

[nltk_data] Downloading package wordnet to /home/user/nltk_data...
[nltk_data] Downloading package punkt_tab to /home/user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /home/user/nltk_data...


{'meteor': 0.5888888888888889}


# LLM Evaluation with MLflow Example Notebook

In this notebook, we will demonstrate how to evaluate various LLMs and RAG systems with MLflow, leveraging simple metrics such as toxicity, as well as LLM-judged metrics such as relevance, and even custom LLM-judged metrics such as professionalism

We need to set our OpenAI API key, since we will be using GPT-4 for our LLM-judged metrics.

In order to set your private key safely, please be sure to either export your key through a command-line terminal for your current instance, or, for a permanent addition to all user-based sessions, configure your favored environment management configuration file (i.e., .bashrc, .zshrc) to have the following entry:

`OPENAI_API_KEY=<your openai API key>`

In [11]:
!pip install mlflow --quiet

In [12]:
import openai
import pandas as pd
import mlflow

## Basic Question-Answering Evaluation

Create a test case of `inputs` that will be passed into the model and `ground_truth` which will be used to compare against the generated output from the model.

In [13]:
eval_df = pd.DataFrame(
    {
        "inputs": [
            "How does useEffect() work?",
            "What does the static keyword in a function mean?",
            "What does the 'finally' block in Python do?",
            "What is the difference between multiprocessing and multithreading?",
        ],
        "ground_truth": [
            "The useEffect() hook tells React that your component needs to do something after render. React will remember the function you passed (we’ll refer to it as our “effect”), and call it later after performing the DOM updates.",
            "Static members belongs to the class, rather than a specific instance. This means that only one instance of a static member exists, even if you create multiple objects of the class, or if you don't create any. It will be shared by all objects.",
            "'Finally' defines a block of code to run when the try... except...else block is final. The finally block will be executed no matter if the try block raises an error or not.",
            "Multithreading refers to the ability of a processor to execute multiple threads concurrently, where each thread runs a process. Whereas multiprocessing refers to the ability of a system to run multiple processors in parallel, where each processor can run one or more threads.",
        ],
    }
)
eval_df

,inputs,ground_truth
0,How does useEffect() work?,The useEffect() hook tells React that your com...
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th..."
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...


Create a simple OpenAI model that asks gpt-4o to answer the question in two sentences. Call `mlflow.evaluate()` with the model and evaluation dataframe. 

In [14]:
from openai import AzureOpenAI
client = AzureOpenAI()

In [15]:
mlflow.set_experiment("LLM Evaluation")

2025/06/27 22:35:06 INFO mlflow.tracking.fluent: Experiment with name 'LLM Evaluation' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///home/user/GAAPB03-training/mlruns/735681311339922250', creation_time=1751043906714, experiment_id='735681311339922250', last_update_time=1751043906714, lifecycle_stage='active', name='LLM Evaluation', tags={}>

In [16]:
import os
os.environ['OPENAI_API_TYPE']='azure'
os.environ['OPENAI_API_KEY']=os.environ['AZURE_OPENAI_API_KEY']
os.environ['OPENAI_API_BASE']=os.environ['AZURE_OPENAI_ENDPOINT']
os.environ['OPENAI_DEPLOYMENT_NAME']='myllm'

In [17]:
with mlflow.start_run() as run:
    system_prompt = "Answer the following question in two sentences"
    basic_qa_model = mlflow.openai.log_model(
        model="myllm",
        task=client.chat.completions,
        artifact_path="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        targets="ground_truth",  # specify which column corresponds to the expected output
        model_type="question-answering",  # model type indicates which metrics are relevant for this task
        evaluators="default",
        extra_metrics=[mlflow.metrics.latency(),
                       mlflow.metrics.flesch_kincaid_grade_level(),
                       mlflow.metrics.ari_grade_level(),
                       mlflow.metrics.rougeL(),mlflow.metrics.bleu()]
    )
results.metrics

2025/06/27 22:35:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/27 22:36:00 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-cfd397014fbf44029de79dde6397be8a
2025/06/27 22:36:00 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/06/27 22:36:00 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-cfd397014fbf44029de79dde6397be8a
2025/06/27 22:36:00 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/06/27 22:36:00 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.
2025/06/27 22:36:00 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2025/06/27 22:36:13 INFO mlflow.mode

{'latency/mean': 3.146435558795929,
 'latency/variance': 1.977233974647742,
 'latency/p90': 4.750580310821533,
 'exact_match/v1': 0.0,
 'rougeL/v1/mean': 0.1937825125175982,
 'rougeL/v1/variance': 0.0012666600050009432,
 'rougeL/v1/p90': 0.2295209934949734,
 'bleu/v1/mean': 0.0,
 'bleu/v1/variance': 0.0,
 'bleu/v1/p90': 0.0}

Inspect the evaluation results table as a dataframe to see row-by-row metrics to further assess model performance

In [18]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,latency,token_count,rougeL/v1/score,bleu/v1/score
0,How does useEffect() work?,The useEffect() hook tells React that your com...,The `useEffect()` hook in React allows you to ...,1.824962,61,0.224719,0
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The `static` keyword in a function signifies t...,1.874156,54,0.172043,0
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python is used in excep...,3.675140,73,0.231579,0
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Multiprocessing involves running multiple proc...,5.211483,77,0.146789,0


## LLM-judged correctness with OpenAI GPT-4

Construct an answer similarity metric using the `answer_similarity()` metric factory function.

In [19]:
from mlflow.metrics.genai import EvaluationExample, answer_similarity

# Create an example to describe what answer_similarity means like for this problem.
example = EvaluationExample(
    input="What is MLflow?",
    output="MLflow is an open-source platform for managing machine "
    "learning workflows, including experiment tracking, model packaging, "
    "versioning, and deployment, simplifying the ML lifecycle.",
    score=4,
    justification="The definition effectively explains what MLflow is "
    "its purpose, and its developer. It could be more concise for a 5-score.",
    grading_context={
        "targets": "MLflow is an open-source platform for managing "
        "the end-to-end machine learning (ML) lifecycle. It was developed by Databricks, "
        "a company that specializes in big data and machine learning solutions. MLflow is "
        "designed to address the challenges that data scientists and machine learning "
        "engineers face when developing, training, and deploying machine learning models."
    },
)

# Construct the metric using OpenAI GPT-4 as the judge
answer_similarity_metric = answer_similarity(model="openai:/gpt-4.1-mini", examples=[example])

print(answer_similarity_metric)

EvaluationMetric(name=answer_similarity, greater_is_better=True, long_name=answer_similarity, version=v1, metric_details=
Task:
You must return the following fields in your response in two lines, one below the other:
score: Your numerical score for the model's answer_similarity based on the rubric
justification: Your reasoning about the model's answer_similarity score

You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called answer_similarity based on the input and output.
A definition of answer_similarity and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them be

Call `mlflow.evaluate()` again but with your new `answer_similarity_metric`

In [20]:
with mlflow.start_run() as run:
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        targets="ground_truth",
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[answer_similarity_metric],  # use the answer similarity metric created above
    )
results.metrics

2025/06/27 22:37:14 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-cfd397014fbf44029de79dde6397be8a
2025/06/27 22:37:14 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/06/27 22:37:14 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2025/06/27 22:37:16 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/06/27 22:37:25 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: RuntimeError('At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.')), skipping metric logging.
2025/06/27 22:37:25 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'toxicity' at index 1 in the `extra_metrics` parameter because it returned None.
20

{'exact_match/v1': 0.0,
 'answer_similarity/v1/mean': 4.0,
 'answer_similarity/v1/variance': 0.0,
 'answer_similarity/v1/p90': 4.0}

See the row-by-row LLM-judged answer similarity score and justifications

In [21]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count,answer_similarity/v1/score,answer_similarity/v1/justification
0,How does useEffect() work?,The useEffect() hook tells React that your com...,The `useEffect()` hook in React allows you to ...,65,4,The output provides a comprehensive explanatio...
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The static keyword in a function indicates tha...,57,4,The output provides a clear explanation of the...
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python is used to defin...,63,4,The output accurately describes the purpose an...
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Multiprocessing involves the use of multiple p...,83,4,The output provides a clear and detailed expla...


## Custom LLM-judged metric for professionalism

Create a custom metric that will be used to determine professionalism of the model outputs. Use `make_genai_metric` with a metric definition, grading prompt, grading example, and judge model configuration

In [22]:
from mlflow.metrics.genai import EvaluationExample, make_genai_metric

professionalism_metric = make_genai_metric(
    name="professionalism",
    definition=(
        "Professionalism refers to the use of a formal, respectful, and appropriate style of communication that is tailored to the context and audience. It often involves avoiding overly casual language, slang, or colloquialisms, and instead using clear, concise, and respectful language"
    ),
    grading_prompt=(
        "Professionalism: If the answer is written using a professional tone, below "
        "are the details for different scores: "
        "- Score 1: Language is extremely casual, informal, and may include slang or colloquialisms. Not suitable for professional contexts."
        "- Score 2: Language is casual but generally respectful and avoids strong informality or slang. Acceptable in some informal professional settings."
        "- Score 3: Language is balanced and avoids extreme informality or formality. Suitable for most professional contexts. "
        "- Score 4: Language is noticeably formal, respectful, and avoids casual elements. Appropriate for business or academic settings. "
        "- Score 5: Language is excessively formal, respectful, and avoids casual elements. Appropriate for the most formal settings such as textbooks. "
    ),
    examples=[
        EvaluationExample(
            input="What is MLflow?",
            output=(
                "MLflow is like your friendly neighborhood toolkit for managing your machine learning projects. It helps you track experiments, package your code and models, and collaborate with your team, making the whole ML workflow smoother. It's like your Swiss Army knife for machine learning!"
            ),
            score=2,
            justification=(
                "The response is written in a casual tone. It uses contractions, filler words such as 'like', and exclamation points, which make it sound less professional. "
            ),
        )
    ],
    version="v1",
    model="openai:/gpt-4o",
    parameters={"temperature": 0.0},
    grading_context_columns=[],
    aggregations=["mean", "variance", "p90"],
    greater_is_better=True,
)

print(professionalism_metric)

EvaluationMetric(name=professionalism, greater_is_better=True, long_name=professionalism, version=v1, metric_details=
Task:
You must return the following fields in your response in two lines, one below the other:
score: Your numerical score for the model's professionalism based on the rubric
justification: Your reasoning about the model's professionalism score

You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called professionalism based on the input and output.
A definition of professionalism and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them before complet

Call `mlflow.evaluate` with your new professionalism metric. 

In [23]:
with mlflow.start_run() as run:
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[professionalism_metric],  # use the professionalism metric we created above
    )
print(results.metrics)

2025/06/27 22:38:10 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-cfd397014fbf44029de79dde6397be8a
2025/06/27 22:38:10 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/06/27 22:38:10 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2025/06/27 22:38:13 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/06/27 22:38:15 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: RuntimeError('At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.')), skipping metric logging.
2025/06/27 22:38:15 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'toxicity' at index 1 in the `extra_metrics` parameter because it returned None.
20

{'professionalism/v1/mean': 4.0, 'professionalism/v1/variance': 0.0, 'professionalism/v1/p90': 4.0}


In [25]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count,professionalism/v1/score,professionalism/v1/justification
0,How does useEffect() work?,The useEffect() hook tells React that your com...,The `useEffect()` hook in React allows you to ...,65,4,"The response is formal and respectful, using c..."
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The `static` keyword in a function indicates t...,70,4,"The response is formal and respectful, using c..."
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python is used to defin...,59,4,"The response is formal and respectful, using c..."
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Multiprocessing involves the use of multiple p...,65,4,"The response is formal and respectful, using c..."


Lets see if we can improve `basic_qa_model` by creating a new model that could perform better by changing the system prompt.

Call `mlflow.evaluate()` using the new model. Observe that the professionalism score has increased!

In [30]:
with mlflow.start_run() as run:
    system_prompt = "Answer the following question using extreme formality."
    professional_qa_model = mlflow.openai.log_model(
        model="myllm",
        task=client.chat.completions,
        artifact_path="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )
    results = mlflow.evaluate(
        professional_qa_model.model_uri,
        eval_df,
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[professionalism_metric],
    )
print(results.metrics)

2025/06/27 22:39:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/27 22:39:40 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-6193e7db52d24b9fbbbbc38da5f5e609
2025/06/27 22:39:40 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/06/27 22:39:40 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-6193e7db52d24b9fbbbbc38da5f5e609
2025/06/27 22:39:40 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/06/27 22:39:40 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2025/06/27 22:39:47 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/06/27 22:39:50 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: RuntimeError('At least one of TensorFlow 2.0 or PyTorch s

{'professionalism/v1/mean': 5.0, 'professionalism/v1/variance': 0.0, 'professionalism/v1/p90': 5.0}


In [31]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count,professionalism/v1/score,professionalism/v1/justification
0,How does useEffect() work?,The useEffect() hook tells React that your com...,"The `useEffect()` hook, a fundamental feature ...",376,5,The response is written in an excessively form...
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...","The use of the keyword ""static"" within the con...",160,5,The response is excessively formal and respect...
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,"In the esteemed programming language Python, t...",230,5,"The response is excessively formal, using prec..."
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,The distinction between multiprocessing and mu...,306,5,"The response is excessively formal, using prec..."
